In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/siim-covid19-detection'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing DateSets

In [ ]:
train_data = pd.read_csv('../input/slim-trained/train_data.csv')

# Extracting Necessary Info

In [ ]:
box = train_data[['id', 'label', 'StudyInstanceUID', 'Path']]

In [ ]:
disease = []
for index in range(len(train_data['Negative for Pneumonia'])):
    for itr in train_data[['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance',
       'Atypical Appearance' ]]:
        if train_data.loc[index, itr] == 1: 
            disease.append(itr)
            
box['Disease'] = disease

In [ ]:
bbox = pd.DataFrame(columns = ['id', 'xmin', 'ymin', 'xmax', 'ymax'])

# Extreacting Bounding Info from Opacity

In [ ]:
for id, value in zip(box['id'], box['label']):           
    if "none" not in value:
        row = [id]
        value = value.split(" ")
        stop = value.count("opacity")
        
        for itr in range (stop):
            xmin = float(value[2])
            row.append(xmin)
            ymin = float(value[3])
            row.append(ymin)
            xmax = float(value[4])
            row.append(xmax)
            ymax = float(value[5])
            row.append(ymax)
            
            index = len(bbox)
            bbox.loc[index] = row
            
            del row[1:]
            del value[0:6]        
    else:
        row = [id, 0, 0, 1, 1]
        index = len(bbox)
        bbox.loc[index] = row    
        row.clear()

In [ ]:
for itr in bbox['id']:
    bbox['id'] = bbox['id'].str.replace('_image', '')

In [ ]:
size_data = pd.read_csv("../input/resized-image-data/image_size.csv")

In [ ]:
for itr in size_data['id']:
    size_data['id'] = size_data['id'].str.replace('.jpg', '')

In [ ]:
df = pd.merge(bbox, size_data, on='id')

In [ ]:
box = box.drop(['label'], axis=1)

In [ ]:
for itr in box['id']:
    box['id'] = box['id'].str.replace('_image', '')

In [ ]:
data = pd.merge(df, box, on='id')

In [ ]:
size = pd.read_csv('../input/image-size/train (1).csv')
size

In [ ]:
x_center =[]
y_center =[]
re_xmax =[]
re_ymax =[]

for index in range(len(size['id'])):
    # need aspect ration how much different between
    x = size.loc[index,'ori_width']
    y = size.loc[index,'ori_height']
    size_x = size.loc[index,'re_width']
    size_y = size.loc[index,'re_height']
    re_x = size_x / x
    re_y = size_y / y
    # resize and Normalize bounding box between 0 and 1
    resize_xmin = size.loc[index,'xmin']* re_x
    resize_ymin = size.loc[index,'ymin']* re_y
    
    
    if size.loc[index,'xmax'] + size.loc[index,'ymax'] == 2: # assume no box 
        x_center.append(0)
        y_center.append(0)
        re_xmax.append(1)        
        re_ymax.append(1)
        
    else: # reagular boxing 
        resize_xmax = size.loc[index,'xmax']* re_x
        x_center.append((resize_xmin + (resize_xmax /2))/416)
        re_xmax.append(resize_xmax/416)     
        resize_ymax = size.loc[index,'ymax']* re_y
        y_center.append((resize_ymin + (resize_ymax /2))/416)
        re_ymax.append(resize_ymax/416)
        

In [ ]:
size['new_xmin'] = x_center
size['new_ymin'] = y_center
size['new_xmax'] = re_xmax
size['new_ymax'] = re_ymax

In [ ]:
size.to_csv('/kaggle/working/train.csv',index=False)